In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from utils import Utils
from utils import spark_utils

import logging

In [2]:
spark = spark_utils.get_spark_session("DEV", "Meli_Datapipeline")

24/08/03 20:40:23 WARN Utils: Your hostname, EPCOBOGW1343 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/03 20:40:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/03 20:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_prints = spark.read.json("data_sources/prints.json")
df_taps = spark.read.json("data_sources/taps.json")
df_pays = spark.read.option("header","True").csv("data_sources/pays.csv")

In [4]:
df_prints.printSchema()

root
 |-- day: string (nullable = true)
 |-- event_data: struct (nullable = true)
 |    |-- position: long (nullable = true)
 |    |-- value_prop: string (nullable = true)
 |-- user_id: long (nullable = true)



In [5]:
df_taps.printSchema()

root
 |-- day: string (nullable = true)
 |-- event_data: struct (nullable = true)
 |    |-- position: long (nullable = true)
 |    |-- value_prop: string (nullable = true)
 |-- user_id: long (nullable = true)



In [6]:
df_pays.printSchema()

root
 |-- pay_date: string (nullable = true)
 |-- total: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- value_prop: string (nullable = true)



In [ ]:
df.show(truncate=False)

In [ ]:
def main(env : str, 
         appName : str
        ):
    
    spark = spark_utils.get_spark_session(env, appName)
    print(type(spark))
    df = spark.read.json("data_sources/prints.json")

In [ ]:
main("DEV", "Meli_Data_Pipeline")